In [2]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px

In [3]:
# Direct Democracy Study

data_path = Path("../data/anes_specialstudy_2012_directdem_dta/anes_specialstudy_2012_directdem_stata12.dta")
data = pd.read_stata(data_path)

First, we need to collect all of the objective questions and find their respective correct answers. Luckily, I've done this and compiled the information in a csv file we can use. 

It appears that there are *three* non-opiniated questions on the survey. These are only the questions I could observe, so there may be more in addition to these. They are:

1. What is Medicare?
2. On which of the following does the U.S. federal government currently spend the least?
3. For how many years is a United States Senator elected – that is, how many years are there in one full term of office for a U.S. Senator?

In [8]:
data["pre_senterm"][0:10]

0    2
1    6
2    4
3    2
4    4
5    2
6    6
7    4
8    4
9    6
Name: pre_senterm, dtype: category
Categories (24, object): [-9. Refused < -4. Error < 1 < 2 ... 50 < 79 < 90 < 99]